In [1]:
import pandas as pd
import numpy as np
from textblob import TextBlob, Word
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords, sentiwordnet as swn
from nltk.stem import WordNetLemmatizer
from nltk import ngrams
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import collections
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
#from stop_words_list import stop_words_list
#from wn_affect import wn_affect 
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [ ]:
pip install WordCloud

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nevin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
import os
os.getcwd()

In [3]:
data = pd.read_csv("Merged_data_without_date.csv")

C:\Users\nevin\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3437: DtypeWarning: Columns (2,3,5,10,12,13,15,16,17,18,19,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
data.head()

In [ ]:
data.isna().sum()

In [ ]:
train, test = train_test_split(data, test_size=0.2)

In [ ]:
len(train.index)

In [ ]:
len(test.index)

# DataPreprocessing

First step is tokenization which converts the text into sentences and the sentences into words. Convert the words into lowercase and remove punctionation

In [4]:
data = data[['review_body']]
data.head()

,review_body
0,My flight from Dulles to Paris was great. My f...
1,"I went for 2 months to Nice, when I arrived i..."
2,"Yesterday Jan 25, we flew Colombo, Sri Lanka t..."
3,First medium-haul experience with Air France....
4,BOD-CDG-ATL: Was having some major issue with...


In [5]:
#case text to lower case, remove punctuation, remove extra whitepsace in the string and on both sides of the string
data['remove_lower_punct'] = data['review_body'].str.lower().str.replace("'", '').str.replace('[^\w\s]', ' ').str.replace(" \d+", " ").str.replace(' +', ' ').str.strip()

data.head(5)

<ipython-input-5-66199223f102>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  data['remove_lower_punct'] = data['review_body'].str.lower().str.replace("'", '').str.replace('[^\w\s]', ' ').str.replace(" \d+", " ").str.replace(' +', ' ').str.strip()


,review_body,remove_lower_punct
0,My flight from Dulles to Paris was great. My f...,my flight from dulles to paris was great my fl...
1,"I went for 2 months to Nice, when I arrived i...",i went for months to nice when i arrived in ni...
2,"Yesterday Jan 25, we flew Colombo, Sri Lanka t...",yesterday jan we flew colombo sri lanka to par...
3,First medium-haul experience with Air France....,first medium haul experience with air france t...
4,BOD-CDG-ATL: Was having some major issue with...,bod cdg atl was having some major issue with d...


In [ ]:
data.columns

In [6]:
data['tokenise'] = data.apply(lambda row: nltk.word_tokenize(str(row[1])), axis=1)
data.head(5)

,review_body,remove_lower_punct,tokenise
0,My flight from Dulles to Paris was great. My f...,my flight from dulles to paris was great my fl...,"[my, flight, from, dulles, to, paris, was, gre..."
1,"I went for 2 months to Nice, when I arrived i...",i went for months to nice when i arrived in ni...,"[i, went, for, months, to, nice, when, i, arri..."
2,"Yesterday Jan 25, we flew Colombo, Sri Lanka t...",yesterday jan we flew colombo sri lanka to par...,"[yesterday, jan, we, flew, colombo, sri, lanka..."
3,First medium-haul experience with Air France....,first medium haul experience with air france t...,"[first, medium, haul, experience, with, air, f..."
4,BOD-CDG-ATL: Was having some major issue with...,bod cdg atl was having some major issue with d...,"[bod, cdg, atl, was, having, some, major, issu..."


In [7]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nevin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [8]:
stop_words = stopwords.words('english')

In [9]:
data['remove_stopwords'] = data['tokenise'].apply(lambda x: [item for item in x if item not in stop_words])

In [10]:
data.head()

,review_body,remove_lower_punct,tokenise,remove_stopwords
0,My flight from Dulles to Paris was great. My f...,my flight from dulles to paris was great my fl...,"[my, flight, from, dulles, to, paris, was, gre...","[flight, dulles, paris, great, flight, attenda..."
1,"I went for 2 months to Nice, when I arrived i...",i went for months to nice when i arrived in ni...,"[i, went, for, months, to, nice, when, i, arri...","[went, months, nice, arrived, nice, baggage, c..."
2,"Yesterday Jan 25, we flew Colombo, Sri Lanka t...",yesterday jan we flew colombo sri lanka to par...,"[yesterday, jan, we, flew, colombo, sri, lanka...","[yesterday, jan, flew, colombo, sri, lanka, pa..."
3,First medium-haul experience with Air France....,first medium haul experience with air france t...,"[first, medium, haul, experience, with, air, f...","[first, medium, haul, experience, air, france,..."
4,BOD-CDG-ATL: Was having some major issue with...,bod cdg atl was having some major issue with d...,"[bod, cdg, atl, was, having, some, major, issu...","[bod, cdg, atl, major, issue, delta, airlines,..."


In [11]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nevin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [12]:
wordnet_lemmatizer = WordNetLemmatizer()

# lemmatise words

data['lemmatise'] = data['remove_stopwords'].apply(lambda x: [wordnet_lemmatizer.lemmatize(y) for y in x if y.isalpha()]) 

data.head(5)

,review_body,remove_lower_punct,tokenise,remove_stopwords,lemmatise
0,My flight from Dulles to Paris was great. My f...,my flight from dulles to paris was great my fl...,"[my, flight, from, dulles, to, paris, was, gre...","[flight, dulles, paris, great, flight, attenda...","[flight, dulles, paris, great, flight, attenda..."
1,"I went for 2 months to Nice, when I arrived i...",i went for months to nice when i arrived in ni...,"[i, went, for, months, to, nice, when, i, arri...","[went, months, nice, arrived, nice, baggage, c...","[went, month, nice, arrived, nice, baggage, ca..."
2,"Yesterday Jan 25, we flew Colombo, Sri Lanka t...",yesterday jan we flew colombo sri lanka to par...,"[yesterday, jan, we, flew, colombo, sri, lanka...","[yesterday, jan, flew, colombo, sri, lanka, pa...","[yesterday, jan, flew, colombo, sri, lanka, pa..."
3,First medium-haul experience with Air France....,first medium haul experience with air france t...,"[first, medium, haul, experience, with, air, f...","[first, medium, haul, experience, air, france,...","[first, medium, haul, experience, air, france,..."
4,BOD-CDG-ATL: Was having some major issue with...,bod cdg atl was having some major issue with d...,"[bod, cdg, atl, was, having, some, major, issu...","[bod, cdg, atl, major, issue, delta, airlines,...","[bod, cdg, atl, major, issue, delta, airline, ..."


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf_vectorizer = TfidfVectorizer()
corpus=[]
for index, row in data.iterrows():
    corpus.append(", ".join(row[24]))
X = tf_vectorizer.fit_transform(corpus)
tf_vectorizer.get_feature_names_out()
X.shape

In [13]:
vectorizer = CountVectorizer(analyzer = 'word', ngram_range = (2, 2))
                            
# join the processed data to be vectorised

vectors = []

for index, row in data.iterrows():
    vectors.append(", ".join(row[4]))

vectorised = vectorizer.fit_transform(vectors)

print(vectorised)
vectorised.shape

  (0, 227272)	1
  (0, 178635)	1
  (0, 410977)	1
  (0, 262168)	1
  (0, 226401)	1
  (0, 47577)	1
  (0, 553491)	1
  (0, 383165)	1
  (0, 243851)	1
  (0, 603087)	1
  (0, 88026)	1
  (0, 143437)	1
  (0, 535938)	1
  (0, 242441)	1
  (0, 332234)	1
  (0, 537444)	1
  (0, 317169)	1
  (0, 498935)	1
  (0, 425494)	1
  (0, 395516)	1
  (0, 341225)	1
  (0, 372272)	1
  (0, 620412)	1
  (0, 235597)	1
  (0, 259439)	1
  :	:
  (34390, 387044)	1
  (34391, 258953)	1
  (34391, 243802)	1
  (34391, 539651)	1
  (34391, 333217)	1
  (34391, 286694)	1
  (34391, 497644)	1
  (34391, 86580)	1
  (34391, 397342)	1
  (34391, 539418)	1
  (34391, 67285)	1
  (34391, 191263)	1
  (34391, 235635)	1
  (34391, 282253)	1
  (34391, 313380)	1
  (34391, 191261)	1
  (34391, 27951)	1
  (34391, 640008)	1
  (34391, 354932)	1
  (34391, 117445)	1
  (34391, 311924)	1
  (34391, 640015)	1
  (34391, 200351)	1
  (34391, 52080)	1
  (34391, 317179)	1


(34392, 643873)

In [14]:
# initisalise LDA Model

lda_model = LatentDirichletAllocation(n_components = 5, # number of topics
                                  random_state = 10,          # random state
                                  evaluate_every = -1,      # compute perplexity every n iters, default: Don't
                                  n_jobs = -1,              # Use all available CPUs
                                 )

lda_output = lda_model.fit_transform(vectorised)

# column names

topic_names = ["Topic" + str(i) for i in range(1, lda_model.n_components + 1)]

# make the pandas dataframe

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns = topic_names)

# get dominant topic for each document

dominant_topic = (np.argmax(df_document_topic.values, axis=1)+1)
df_document_topic['Dominant_topic'] = dominant_topic

# join to original dataframes

data = pd.merge(data, df_document_topic, left_index = True, right_index = True, how = 'outer')
data['Dominant_topic']=data['Dominant_topic'].astype(int)
display(data.head(10))

,review_body,remove_lower_punct,tokenise,remove_stopwords,lemmatise,Topic1,Topic2,Topic3,Topic4,Topic5,Dominant_topic
0,My flight from Dulles to Paris was great. My f...,my flight from dulles to paris was great my fl...,"[my, flight, from, dulles, to, paris, was, gre...","[flight, dulles, paris, great, flight, attenda...","[flight, dulles, paris, great, flight, attenda...",0.01,0.01,0.01,0.98,0.01,4
1,"I went for 2 months to Nice, when I arrived i...",i went for months to nice when i arrived in ni...,"[i, went, for, months, to, nice, when, i, arri...","[went, months, nice, arrived, nice, baggage, c...","[went, month, nice, arrived, nice, baggage, ca...",0.01,0.01,0.96,0.01,0.01,3
2,"Yesterday Jan 25, we flew Colombo, Sri Lanka t...",yesterday jan we flew colombo sri lanka to par...,"[yesterday, jan, we, flew, colombo, sri, lanka...","[yesterday, jan, flew, colombo, sri, lanka, pa...","[yesterday, jan, flew, colombo, sri, lanka, pa...",0.00,0.00,0.98,0.00,0.00,3
3,First medium-haul experience with Air France....,first medium haul experience with air france t...,"[first, medium, haul, experience, with, air, f...","[first, medium, haul, experience, air, france,...","[first, medium, haul, experience, air, france,...",0.00,1.00,0.00,0.00,0.00,2
4,BOD-CDG-ATL: Was having some major issue with...,bod cdg atl was having some major issue with d...,"[bod, cdg, atl, was, having, some, major, issu...","[bod, cdg, atl, major, issue, delta, airlines,...","[bod, cdg, atl, major, issue, delta, airline, ...",0.00,0.99,0.00,0.00,0.00,2
5,"My first flight was cancelled last minute, ca...",my first flight was cancelled last minute caus...,"[my, first, flight, was, cancelled, last, minu...","[first, flight, cancelled, last, minute, causi...","[first, flight, cancelled, last, minute, causi...",0.99,0.00,0.00,0.00,0.00,1
6,I booked with KLM which was a nightmare. I was...,i booked with klm which was a nightmare i was ...,"[i, booked, with, klm, which, was, a, nightmar...","[booked, klm, nightmare, supposed, fly, air, f...","[booked, klm, nightmare, supposed, fly, air, f...",0.99,0.00,0.00,0.00,0.00,1
7,I booked my trip 2 months in advance. This ai...,i booked my trip months in advance this airlin...,"[i, booked, my, trip, months, in, advance, thi...","[booked, trip, months, advance, airline, cance...","[booked, trip, month, advance, airline, cancel...",0.00,0.99,0.00,0.00,0.00,2
8,Frankfurt - Toronto via Paris CDG. My first l...,frankfurt toronto via paris cdg my first long ...,"[frankfurt, toronto, via, paris, cdg, my, firs...","[frankfurt, toronto, via, paris, cdg, first, l...","[frankfurt, toronto, via, paris, cdg, first, l...",0.00,0.00,0.99,0.00,0.00,3
9,Frankfurt - Toronto via Paris CDG. My first lo...,frankfurt toronto via paris cdg my first long ...,"[frankfurt, toronto, via, paris, cdg, my, firs...","[frankfurt, toronto, via, paris, cdg, first, l...","[frankfurt, toronto, via, paris, cdg, first, l...",0.00,0.00,0.99,0.00,0.00,3


In [ ]:
''''# initisalise LDA Model on TF IDF

lda_model2 = LatentDirichletAllocation(n_components = 5, # number of topics
                                  random_state = 10,          # random state
                                  evaluate_every = -1,      # compute perplexity every n iters, default: Don't
                                  n_jobs = -1,              # Use all available CPUs
                                 )

lda_output2 = lda_model2.fit_transform(X)

# column names

topic_names2 = ["Topic_Tf_" + str(i) for i in range(1, lda_model.n_components + 1)]

# make the pandas dataframe

df_document_topic2 = pd.DataFrame(np.round(lda_output2, 2), columns = topic_names2)

# get dominant topic for each document

dominant_topic_tf = (np.argmax(df_document_topic2.values, axis=1)+1)
df_document_topic2['Dominant_topic_tf'] = dominant_topic_tf

# join to original dataframes

data = pd.merge(data, df_document_topic2, left_index = True, right_index = True, how = 'outer')
display(data.head(10))
'''

In [ ]:
# apply sentiment analysis
analyser = SentimentIntensityAnalyzer()

sentiment_score_list = []
sentiment_label_list = []

for i in data['remove_lower_punct'].values.tolist():
    sentiment_score = analyser.polarity_scores(i)

    if sentiment_score['compound'] >= 0.05:
        sentiment_score_list.append(sentiment_score['compound'])
        sentiment_label_list.append('Positive')
    elif sentiment_score['compound'] > -0.05 and sentiment_score['compound'] < 0.05:
        sentiment_score_list.append(sentiment_score['compound'])
        sentiment_label_list.append('Neutral')
    elif sentiment_score['compound'] <= -0.05:
        sentiment_score_list.append(sentiment_score['compound'])
        sentiment_label_list.append('Negative')
    
data['sentiment'] = sentiment_label_list
data['sentiment score'] = sentiment_score_list

data.head()

In [15]:
# Topic-Keyword Matrix
df_topic_keywords = pd.DataFrame(lda_model.components_)

# Assign Column and Index
df_topic_keywords.columns = vectorizer.get_feature_names()
df_topic_keywords.index = topic_names

df_topic_no = pd.DataFrame(df_topic_keywords.idxmax())
df_scores = pd.DataFrame(df_topic_keywords.max())

tmp = pd.merge(df_topic_no, df_scores, left_index=True, right_index=True)
tmp.columns = ['topic', 'relevance_score']

display(tmp)

,topic,relevance_score
aa aa,Topic1,1.200039
aa absolutely,Topic2,1.199997
aa account,Topic1,1.199999
aa actually,Topic5,1.199998
aa adding,Topic3,1.199995
...,...,...
每位空乘人员见到旅客都会非常热情的说 您好,Topic3,1.199999
登机时 每位空乘人员见到旅客都会非常热情的说,Topic3,1.199999
空乘人员并没有因为疫情的原因对我们敬而远之 反而以更加热情的态度对待我们,Topic3,1.199999
第一入境地为西安 在这个疫情严重的特殊时期,Topic3,1.199999


In [16]:
all_topics = []

for i in tmp['topic'].unique():    
    tmp_1 = tmp.loc[tmp['topic'] == i].reset_index()
    tmp_1 = tmp_1.sort_values('relevance_score', ascending=False).head(1)

    tmp_2 = []
    tmp_2.append(tmp_1['topic'].unique()[0])
    tmp_2.append(list(tmp_1['index'].unique()))
    all_topics.append(tmp_2)

all_topics = pd.DataFrame(all_topics, columns=['Dominant_topic', 'topic_name'])
display(all_topics)

,Dominant_topic,topic_name
0,Topic1,[premium economy]
1,Topic2,[seat front]
2,Topic5,[business class]
3,Topic3,[cabin crew]
4,Topic4,[customer service]


In [ ]:
import altair as alt
alt.Chart(data).mark_bar().encode(
    x='Dominant_topic:N',
    y='count():Q',
    color="sentiment:N"
)

In [ ]:
!pip install pyldavis